importer données rotten tomatoes

In [1]:
import numpy as np
import torch
import torchvision
import unicodedata
import string
import re
import random
import pickle as pkl
from torch import nn
from torch.autograd import Variable


#load RottenTomatoes sentiment analysis dataset
def load_tsv(filename):
    phID, stcID, ph, sentiment = [],[],[],[]
    with open(filename) as f: 
        l = f.readline()
        if (len(l.split("\t")) == 4):
            '''TRAIN dataset'''
            for line in f:
                l = line.split("\t")
                if (int(l[3][0]) < 2):
                    phID.append(l[0]), stcID.append(l[1]), ph.append(l[2]), sentiment.append(0)
                else:
                    phID.append(l[0]), stcID.append(l[1]), ph.append(l[2]), sentiment.append(1)
        else:
            for line in f:
                l = line.split("\t")
                phID.append(l[0]), stcID.append(l[1]), ph.append(l[2])
                
    return np.array(phID).astype(int), np.array(stcID).astype(int), np.array(ph), np.array(sentiment).astype(int)

#make validation dataset
def split_train_test(data, percentage=80):
    size = len(data)
    indexes = np.arange(0,size)
    np.random.shuffle(indexes)
    
    train_indexes = indexes[0:int(size*percentage/100)]
    test_indexes = indexes[int(size*percentage/100):-1]
    
    return train_indexes, test_indexes
    
phraseID, sentenceID, sentences, sentiment = load_tsv("SKIM-RNN/train.tsv")
train_indexes, test_indexes = split_train_test(phraseID)

Module de preprocessing

In [2]:
'''Trim, Store, Count, Index words from dataset'''

SOS_token = 0
EOS_token = 1

class Preprocesser():
    
    '''main functions'''
    def __init__(self,corpus):
        '''corpus : np_array(string)'''
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
        self.corpus = corpus
        self.size = len(corpus)
    
    # Lowercase, trim, and remove non-letter characters
    # (no stop words in skim rnn ?)
    def normalize(self):
        new_corpus = np.array([])
        steps,i = np.arange(0,self.size,self.size/10),0
        for s in self.corpus:
            uni_s = s.tostring().decode('unicode-escape')
            uni_s = self.unicodeToAscii(uni_s.lower().strip())
            uni_s = re.sub(r"([.!?])", r" \1", uni_s)
            uni_s = re.sub(r"[^a-zA-Z.!?]+", r" ", uni_s)
            new_corpus = np.append(new_corpus, uni_s)
            if (i in steps):
                print ("...")
            i+=1
        self.corpus = new_corpus


    def addSentences(self):
        for sentence in self.corpus:
            self.addSentence(sentence)
        
    '''called within the module:'''
    def unicodeToAscii(self,s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def save(self,filename):
        pkl.dump(self.corpus,open(filename+"_corpus.pkl",'wb'))
        pkl.dump(self.word2count,open(filename+"_w2c.pkl",'wb'))
        pkl.dump(self.word2index,open(filename+"_w2i.pkl",'wb'))
        pkl.dump(self.index2word,open(filename+"_i2w.pkl", 'wb'))
    
    def load(self, filename):
        self.corpus = pkl.load(open(filename+"_corpus.pkl", 'rb'))
        self.word2count = pkl.load(open(filename+"_w2c.pkl", 'rb'))
        self.word2index = pkl.load(open(filename+"_w2i.pkl", 'rb'))
        self.index2word = pkl.load(open(filename+"_i2w.pkl", 'rb'))
        self.size = len(self.corpus)
        self.n_words = len(self.word2count.keys())
        
 

Fonctions pour la gestion des inputs 

In [3]:
def binarize(data, n_words, neg_value=0):
    """
    encode target en one-hot, si neg_value 
    vaut zéro, ou en -1/1, si neg_value vaut
    -1 par exemple
    """
    y_onehot = torch.FloatTensor(n_words)
    y_onehot.zero_().add_(neg_value)
    return y_onehot.scatter_(0, data, 1)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = torch.LongTensor(indexes).view(-1, 1)
    return result


def makeInputTarget(lang, sentence, target, n_classes=2):
    input_variable = variableFromSentence(lang, sentence)
    if target >= n_classes:
        print 'target not in range (0, #classes - 1)'
        return -1
    target_variable = Variable(torch.LongTensor([target]))
    return (input_variable, target_variable)


In [4]:
#preprocesser = Preprocesser(sentences)
#preprocesser.normalize()
#preprocesser.addSentences()

#preprocesser.save("preprocessing_IMDB")


preprocesser = Preprocesser({})
preprocesser.load("preprocessing_IMDB")

Modules RNN

In [69]:
'''RNN module : embedding layer -> gru layer -> linear layer (for output classification) -> softmax'''


class Selector(nn.Module):
    '''Linear transformation and logsoftmax for the RNN choice'''
    def __init__(self, input_size, n_choices=2):
        super(Selector, self).__init__()
        self.input_size = input_size
        self.n_choices = n_choices
        self.linear = nn.Linear(input_size, n_choices)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, x):
        return self.softmax(self.linear(x))
    

class RNN(nn.Module):
    '''Standard RNN with gru'''
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size)
        
        self.h20 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.h20(output)
        output = self.softmax(output.view(1,-1))
        return output, hidden
    
    def initHidden(self):
        return Variable(torch.rand(1, 1, self.hidden_size))
    
    
    
class SkimRNN(nn.Module):
    '''Skim RNN model: a selector and two RNNs modules'''
    def __init__(self, input_size, embedding_size, d, dprime, output_size, k  ):
        super(SkimRNN, self).__init__()
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        self.selector = Selector(embedding_size + d, n_choices=k)
        self.mainRNN = RNN(embedding_size, d, output_size)
        self.smallRNN = RNN(embedding_size, dprime, output_size)

        self.d = d
        self.dprime = dprime
        
        self.skimcount = 0
        self.count = 0
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        input = torch.cat( (embedded, hidden), 2).view(1,-1)
        p = self.selector(input)
        q = p.exp().multinomial()
        if q.data[0,0]==0:
            #main RNN
            output, hidden = self.mainRNN(embedded, hidden)
        else:
            #small RNN
            
            # ce n'est plus l'état caché entier qui passe dans le petit RNN, mais le slice sur les dprime premiers
            # élements. Pas sûr de mon coup mais je vois pas comment ça peut marcher avec les dimensions si on a
            # un état caché de taille d en entrée dans le petit RNN. 
            # hidden décomposé en h0 (dprime premiers éléments) et h1
            
            h0 = hidden.view(-1)[:self.dprime]
            output, h0 = self.smallRNN(embedded, h0.view(1,1,-1))
            h1 = hidden.view(-1)[self.dprime:]
            hidden = torch.cat( (h0.view(1,1,-1), h1.view(1,1,-1)), 2)
            self.skimcount += 1
        self.count += 1
        return output, p, hidden
        
    def initHidden(self):
        return Variable(torch.rand(1, 1, self.d))
    
        
    
    

Fonctions pour l'apprentissage

In [95]:
def gumbel():
    return -torch.log(-torch.log(torch.FloatTensor(2).uniform_()))

def r(logp, g, temperature):
    num = [torch.exp( (pi + gi)/temperature) for (pi,gi) in zip(logp,g)]
    denum = torch.sum(torch.exp( (logp.data + g)/temperature))
    return [n.data/denum for n in num]


def train(preprocesser, input,target,rnn, optimizers, criterion, temperature=0.5, gamma=1e-2):
    '''todo : commenter les inputs'''
    for o in optimizers:
        o.zero_grad()
        
    hidden = rnn.initHidden()

    input_length = input.size()[0]
    logp = []
    rp = []
    loss = 0
    loss_output, loss_linear = 0, 0
    
    for t in range(input_length):
        x = input[t]
        output, p, hidden = rnn(Variable(x), hidden)
        logp.append(p.data)
        g = gumbel()    
        rp.append(r(p,g,temperature))
    
        c = criterion(output.view(1,-1), target)
        loss = criterion(output.view(1,-1), target).data 
        loss_output += loss
        loss_linear += loss * rp[t][0].prod()
    logp_skim = [l[0,1] for l in logp]   #get logp probabibilites of skimming for each word
    Variable(loss_output, requires_grad=True).backward()
    loss_linear = Variable( loss_linear +(gamma/input_length) * np.sum(logp_skim), requires_grad=True)
    loss_linear.backward()

#     for p in rnn.parameters():
#         print p
#     for o in optimizers:
#         print o
#         o.step()
#     for p in rnn.parameters():
#         print p
    print ber
    return output ,loss_output[0]





Hyperparamètres + apprentissage

In [96]:
from torch import optim
rnn =  SkimRNN(preprocesser.n_words, 512, 200, 5, 2, 2)
criterion = nn.NLLLoss()


optimizer_rnn = optim.Adam([{'params':rnn.smallRNN.parameters()},{'params':rnn.mainRNN.parameters()}], lr=1e-2)
optimizer_linear = optim.SGD(rnn.selector.parameters(), lr=1e-2)

epoch,mod = 0,5
losses = []
accs = []

for epoch in range(100):
    for i in train_indexes[:500]:
        s,t =preprocesser.corpus[i], sentiment[i]
        input,target =  makeInputTarget(preprocesser, s, t)
        output, l = train(preprocesser,input,target,rnn,[optimizer_rnn, optimizer_linear],criterion,0.5)

        

    if epoch%mod == 0:
        losses.append(l)
        accs.append(accuracy(test_indexes[:200],preprocesser,rnn))
        print epoch


NameError: global name 'ber' is not defined

# TODO

- step qui màj les paramètres (backward graph?)
          
- inputs via glove


- vérifier résultats après apprentissage


- faire des batchs


- implémenter skim rnn avec option lstm


- implémenter fonctions d'évaluations


In [63]:
def accuracy(indexes,preprocesser,rnn):
    acc = 0.0
    for i in indexes:
        s,t =preprocesser.corpus[i], sentiment[i]
        input,target =  makeInputTarget(preprocesser, s, t)
        input_length = input.size()[0]
        
        for word in range(input_length):
            hidden = rnn.initHidden()
            x = input[word]
            output, p, hidden = rnn(Variable(x), hidden)  
        if output.exp().multinomial().data[0,0] == target.data[0]:
            acc += 1
            
    return 100*float(acc/len(indexes))

In [87]:
accs

[47.5,
 51.0,
 55.50000000000001,
 56.99999999999999,
 55.50000000000001,
 54.50000000000001,
 52.0,
 48.0,
 58.5,
 50.5,
 55.00000000000001,
 52.5,
 48.5,
 48.0,
 47.0,
 56.00000000000001,
 45.0,
 53.0,
 55.50000000000001,
 59.0]

In [86]:
1.0*rnn.skimcount/rnn.count

0.5166776921986486